In [84]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
import tensorflow as tf

In [85]:
df = pd.read_csv('mabels.csv')
df.head()

,filepath,xmin,xmax,ymin,ymax
0,./images\N1.xml,1093,1396,645,727
1,./images\N100.xml,134,301,312,350
2,./images\N101.xml,31,139,128,161
3,./images\N102.xml,164,316,216,243
4,./images\N103.xml,813,1067,665,724


In [86]:
import xml.etree.ElementTree as xet

In [87]:
def getfilename(filename):
    filename_image = xet.parse(filename).getroot().find("filename").text
    file_path_image = os.path.join('./images',filename_image)
    return file_path_image

In [88]:
image_path = list(df['filepath'].apply(getfilename))

In [89]:
image_path[0]

'./images\\N1.jpeg'

### READ AN IMAGE

In [90]:
my_image = cv2.imread(image_path[0])
#build the bounding box (which is our output)
cv2.rectangle(my_image,(1093,645),(1396,727),(255,245, 0),3)
#show image
cv2.namedWindow('example',cv2.WINDOW_NORMAL) #it allows to resize the window
cv2.imshow('example',my_image)
cv2.waitKey(0) #allows to display a window for given milliseconds or until any key is pressed
cv2.destroyAllWindows() #if i press any button the window will be closed

### Data Preprocessing

In [91]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [92]:
labels = df.iloc[:,1:].values

In [99]:
data = []
output = []
for ind in range(len(image_path)):
    image = image_path[ind]
    img_arr = cv2.imread(image)
    h,w,d = img_arr.shape
    # prepprocesing
    load_image = load_img(image,target_size=(224,224))
    load_image_arr = img_to_array(load_image)
    norm_load_image_arr = load_image_arr/255.0 # normalization
    # normalization to labels
    xmin,xmax,ymin,ymax = labels[ind]
    nxmin,nxmax = xmin/w,xmax/w
    nymin,nymax = ymin/h,ymax/h
    label_norm = (nxmin,nxmax,nymin,nymax) # normalized output
    # -------------- append
    data.append(norm_load_image_arr)
    output.append(label_norm)

### SPLIT DATA

In [100]:
#convert data into a numpy array
X = np.array(data,dtype=np.float32)
y = np.array(output,dtype=np.float32)

In [101]:
x_train,x_test,y_train,y_test = train_test_split(X,y,train_size=0.8,random_state=0)
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((180, 224, 224, 3), (45, 224, 224, 3), (180, 4), (45, 4))

### Deep Learning Model

In [102]:
from tensorflow.keras.applications import MobileNetV2,InceptionV3,InceptionResNetV2
from tensorflow.keras.layers import Dense,Dropout,Flatten,Input
from tensorflow.keras.models import Model
import tensorflow as tf

In [103]:
inception_resnet = InceptionResNetV2(weights="imagenet",include_top=False,
                                     input_tensor=Input(shape=(224,224,3)))
inception_resnet.trainable=False
# ---------------------
headmodel = inception_resnet.output
headmodel = Flatten()(headmodel)
headmodel = Dense(500,activation="relu")(headmodel)
headmodel = Dense(250,activation="relu")(headmodel)
headmodel = Dense(4,activation='sigmoid')(headmodel)
# ---------- model
model = Model(inputs=inception_resnet.input,outputs=headmodel)
# complie model
model.compile(loss='mse',optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4))
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_1018 (Conv2D)           (None, 111, 111, 32  864         ['input_7[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_1018 (Batc  (None, 111, 111, 32  96         ['conv2d_1018[0][0]']            
 hNormalization)                )                                                           

                                                                                                  
 conv2d_1025 (Conv2D)           (None, 25, 25, 64)   76800       ['activation_1024[0][0]']        
                                                                                                  
 conv2d_1028 (Conv2D)           (None, 25, 25, 96)   82944       ['activation_1027[0][0]']        
                                                                                                  
 conv2d_1029 (Conv2D)           (None, 25, 25, 64)   12288       ['average_pooling2d_5[0][0]']    
                                                                                                  
 batch_normalization_1023 (Batc  (None, 25, 25, 96)  288         ['conv2d_1023[0][0]']            
 hNormalization)                                                                                  
                                                                                                  
 batch_nor

                                                                                                  
 block35_1_mixed (Concatenate)  (None, 25, 25, 128)  0           ['activation_1030[0][0]',        
                                                                  'activation_1032[0][0]',        
                                                                  'activation_1035[0][0]']        
                                                                                                  
 block35_1_conv (Conv2D)        (None, 25, 25, 320)  41280       ['block35_1_mixed[0][0]']        
                                                                                                  
 block35_1 (Lambda)             (None, 25, 25, 320)  0           ['mixed_5b[0][0]',               
                                                                  'block35_1_conv[0][0]']         
                                                                                                  
 block35_1

 batch_normalization_1043 (Batc  (None, 25, 25, 32)  96          ['conv2d_1043[0][0]']            
 hNormalization)                                                                                  
                                                                                                  
 batch_normalization_1046 (Batc  (None, 25, 25, 48)  144         ['conv2d_1046[0][0]']            
 hNormalization)                                                                                  
                                                                                                  
 activation_1043 (Activation)   (None, 25, 25, 32)   0           ['batch_normalization_1043[0][0]'
                                                                 ]                                
                                                                                                  
 activation_1046 (Activation)   (None, 25, 25, 48)   0           ['batch_normalization_1046[0][0]'
          

 batch_normalization_1053 (Batc  (None, 25, 25, 64)  192         ['conv2d_1053[0][0]']            
 hNormalization)                                                                                  
                                                                                                  
 activation_1048 (Activation)   (None, 25, 25, 32)   0           ['batch_normalization_1048[0][0]'
                                                                 ]                                
                                                                                                  
 activation_1050 (Activation)   (None, 25, 25, 32)   0           ['batch_normalization_1050[0][0]'
                                                                 ]                                
                                                                                                  
 activation_1053 (Activation)   (None, 25, 25, 64)   0           ['batch_normalization_1053[0][0]'
          

...

                                                                                                  
 batch_normalization_1187 (Batc  (None, 5, 5, 224)   672         ['conv2d_1187[0][0]']            
 hNormalization)                                                                                  
                                                                                                  
 activation_1187 (Activation)   (None, 5, 5, 224)    0           ['batch_normalization_1187[0][0]'
                                                                 ]                                
                                                                                                  
 conv2d_1185 (Conv2D)           (None, 5, 5, 192)    399360      ['block8_1_ac[0][0]']            
                                                                                                  
 conv2d_1188 (Conv2D)           (None, 5, 5, 256)    172032      ['activation_1187[0][0]']        
          

 conv2d_1195 (Conv2D)           (None, 5, 5, 224)    129024      ['activation_1194[0][0]']        
                                                                                                  
 batch_normalization_1195 (Batc  (None, 5, 5, 224)   672         ['conv2d_1195[0][0]']            
 hNormalization)                                                                                  
                                                                                                  
 activation_1195 (Activation)   (None, 5, 5, 224)    0           ['batch_normalization_1195[0][0]'
                                                                 ]                                
                                                                                                  
 conv2d_1193 (Conv2D)           (None, 5, 5, 192)    399360      ['block8_3_ac[0][0]']            
                                                                                                  
 conv2d_11

                                                                                                  
 conv2d_1203 (Conv2D)           (None, 5, 5, 224)    129024      ['activation_1202[0][0]']        
                                                                                                  
 batch_normalization_1203 (Batc  (None, 5, 5, 224)   672         ['conv2d_1203[0][0]']            
 hNormalization)                                                                                  
                                                                                                  
 activation_1203 (Activation)   (None, 5, 5, 224)    0           ['batch_normalization_1203[0][0]'
                                                                 ]                                
                                                                                                  
 conv2d_1201 (Conv2D)           (None, 5, 5, 192)    399360      ['block8_5_ac[0][0]']            
          

                                                                 ]                                
                                                                                                  
 conv2d_1211 (Conv2D)           (None, 5, 5, 224)    129024      ['activation_1210[0][0]']        
                                                                                                  
 batch_normalization_1211 (Batc  (None, 5, 5, 224)   672         ['conv2d_1211[0][0]']            
 hNormalization)                                                                                  
                                                                                                  
 activation_1211 (Activation)   (None, 5, 5, 224)    0           ['batch_normalization_1211[0][0]'
                                                                 ]                                
                                                                                                  
 conv2d_12

 activation_1218 (Activation)   (None, 5, 5, 192)    0           ['batch_normalization_1218[0][0]'
                                                                 ]                                
                                                                                                  
 conv2d_1219 (Conv2D)           (None, 5, 5, 224)    129024      ['activation_1218[0][0]']        
                                                                                                  
 batch_normalization_1219 (Batc  (None, 5, 5, 224)   672         ['conv2d_1219[0][0]']            
 hNormalization)                                                                                  
                                                                                                  
 activation_1219 (Activation)   (None, 5, 5, 224)    0           ['batch_normalization_1219[0][0]'
                                                                 ]                                
          

### Train the model

In [104]:
from tensorflow.keras.callbacks import TensorBoard

In [105]:
tfb = TensorBoard('object_detection')

In [107]:
history = model.fit(x=x_train,y=y_train,batch_size=10,epochs=100,
                    validation_data=(x_test,y_test),callbacks=[tfb])

Epoch 1/100
18/18 [==============================] - 29s 2s/step - loss: 0.0360 - val_loss: 0.0389
Epoch 2/100
18/18 [==============================] - 31s 2s/step - loss: 0.0357 - val_loss: 0.0393
Epoch 3/100
18/18 [==============================] - 28s 2s/step - loss: 0.0365 - val_loss: 0.0386
Epoch 4/100
18/18 [==============================] - 31s 2s/step - loss: 0.0353 - val_loss: 0.0385
Epoch 5/100
18/18 [==============================] - 31s 2s/step - loss: 0.0344 - val_loss: 0.0388
Epoch 6/100
18/18 [==============================] - 27s 2s/step - loss: 0.0344 - val_loss: 0.0393
Epoch 7/100
18/18 [==============================] - 29s 2s/step - loss: 0.0345 - val_loss: 0.0426
Epoch 8/100
18/18 [==============================] - 34s 2s/step - loss: 0.0350 - val_loss: 0.0404
Epoch 9/100
18/18 [==============================] - 32s 2s/step - loss: 0.0349 - val_loss: 0.0407
Epoch 10/100
18/18 [==============================] - 33s 2s/step - loss: 0.0350 - val_loss: 0.0379
Epoch 11/

18/18 [==============================] - 26s 1s/step - loss: 9.9859e-05 - val_loss: 0.0062
Epoch 83/100
18/18 [==============================] - 26s 1s/step - loss: 1.0142e-04 - val_loss: 0.0063
Epoch 84/100
18/18 [==============================] - 27s 2s/step - loss: 1.0950e-04 - val_loss: 0.0065
Epoch 85/100
18/18 [==============================] - 27s 1s/step - loss: 8.1954e-05 - val_loss: 0.0064
Epoch 86/100
18/18 [==============================] - 27s 1s/step - loss: 9.2715e-05 - val_loss: 0.0063
Epoch 87/100
18/18 [==============================] - 26s 1s/step - loss: 1.2101e-04 - val_loss: 0.0062
Epoch 88/100
18/18 [==============================] - 26s 1s/step - loss: 1.2914e-04 - val_loss: 0.0065
Epoch 89/100
18/18 [==============================] - 27s 1s/step - loss: 1.4268e-04 - val_loss: 0.0064
Epoch 90/100
18/18 [==============================] - 26s 1s/step - loss: 1.8624e-04 - val_loss: 0.0061
Epoch 91/100
18/18 [==============================] - 27s 2s/step - loss: 1.9

In [108]:
model.save("./models/object_detection_model.h5")